## Notebook to scrape, download, and preprocess APOD images

In [ ]:
import glob
from bs4 import BeautifulSoup
import requests
from PIL import Image
from io import BytesIO

In [ ]:
files = glob.glob('*.html')
files

In [ ]:
url = files[1]

soup = BeautifulSoup(open(url, encoding="utf8"), "html.parser")

img_links = [x.find_previous('a') for x in soup('img')]
img_refs = [x['href'] for x in img_links if x]

In [ ]:
len(img_refs)

In [ ]:
r = requests.get(img_refs[0])

im_soup = BeautifulSoup(r.content, "html.parser")

In [ ]:
url_stem = 'https://apod.nasa.gov/apod/'
pic_url = url_stem + im_soup('img')[0].attrs['src']

In [ ]:
response = requests.get(pic_url)
img = Image.open(BytesIO(response.content))

In [ ]:
display(img)